In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os

propublica_token = os.environ['propublica_token']

memberurl = 'https://voteview.com/static/data/out/members/HS117_members.csv'
cvoteurl = 'https://voteview.com/static/data/out/rollcalls/HS117_rollcalls.csv'
memvoteurl = 'https://voteview.com/static/data/out/votes/HS117_votes.csv'
cpartyurl = 'https://voteview.com/static/data/out/parties/HS117_parties.csv'

members_vv = pd.read_csv(memberurl)
cvote_vv = pd.read_csv(cvoteurl)
memvote_vv = pd.read_csv(memvoteurl)
cparty_vv = pd.read_csv(cpartyurl)

useragent_url = 'https://httpbin.org/user-agent'
r = requests.get(useragent_url)
useragent = json.loads(r.text)['user-agent']

headers = {'X-API-Key': propublica_token,
          'User-Agent': useragent,
          'From': 'jkropko@virginia.edu'}

root = 'https://api.propublica.org'
congress = '117'
chamber = 'house'
memberendpoint = '/congress/v1/{congress}/{chamber}/members.json'.format(congress=congress, chamber=chamber)

r = requests.get(root + memberendpoint, headers = headers)
myjson = json.loads(r.text)
house_pp = pd.json_normalize(myjson, record_path = ['results', 'members'])

chamber = 'senate'
memberendpoint = '/congress/v1/{congress}/{chamber}/members.json'.format(congress=congress, chamber=chamber)
r = requests.get(root + memberendpoint, headers = headers)
myjson = json.loads(r.text)
senate_pp = pd.json_normalize(myjson, record_path = ['results', 'members'])

billsendpoint = '/congress/v1/{congress}/{chamber}/bills/{billtype}.json'.format(congress='117',
                                                                             chamber='both',
                                                                             billtype='introduced')
n = 20
i = 0
bills_117 = pd.DataFrame()
while n == 20:
    myparams = {'query': '', 'offset':i}
    r = requests.get(root + billsendpoint, headers = headers, params = myparams)
    myjson = json.loads(r.text)
    bills_pp = pd.json_normalize(myjson, record_path = ['results', 'bills'])
    bills_117 = pd.concat([bills_117, bills_pp], ignore_index=True)
    i = i + 20
    print(i)
    n = myjson['results'][0]['num_results']

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 600)

In [3]:
members_vv.sort_values('nominate_dim1', ascending=True)

,congress,chamber,icpsr,state_icpsr,district_code,state_abbrev,party_code,occupancy,last_means,bioname,bioguide_id,born,died,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,conditional,nokken_poole_dim1,nokken_poole_dim2
395,117,House,21922,49,29,TX,100,NaN,NaN,"GARCIA, Sylvia",G000587,1950,NaN,-0.831,0.504,-47.27894,0.93653,721.0,21.0,NaN,-0.890,0.457
496,117,Senate,41301,3,0,MA,100,NaN,NaN,"WARREN, Elizabeth",W000817,1949,NaN,-0.753,-0.327,-23.89642,0.97089,809.0,9.0,NaN,-0.667,-0.431
461,117,Senate,41701,71,0,CA,100,NaN,NaN,"HARRIS, Kamala Devi",H001075,1964,NaN,-0.709,-0.105,-0.00037,0.99981,2.0,0.0,NaN,0.000,0.000
75,117,House,29778,71,13,CA,100,NaN,NaN,"LEE, Barbara",L000551,1946,NaN,-0.681,-0.585,-13.84128,0.98088,717.0,4.0,NaN,-0.508,-0.595
436,117,House,21726,73,7,WA,100,NaN,NaN,"JAYAPAL, Pramila",J000298,1965,NaN,-0.681,-0.358,-16.83215,0.97696,722.0,6.0,NaN,-0.472,-0.643
149,117,House,21906,21,6,IL,100,NaN,NaN,"CASTEN, Sean",C001117,1971,NaN,-0.657,0.595,-66.11019,0.91012,702.0,21.0,NaN,-0.822,0.569
69,117,House,29106,71,43,CA,100,NaN,NaN,"WATERS, Maxine",W000187,1938,NaN,-0.656,-0.076,-45.51903,0.93756,706.0,18.0,NaN,-0.885,-0.021
118,117,House,22163,43,20,FL,100,NaN,NaN,"CHERFILUS-MCCORMICK, Sheila",C001127,1979,NaN,-0.629,0.470,-10.45086,0.96919,334.0,7.0,NaN,-0.629,0.469
256,117,House,21538,12,12,NJ,100,NaN,NaN,"WATSON COLEMAN, Bonnie",W000822,1945,NaN,-0.617,-0.175,-19.31259,0.97371,725.0,7.0,NaN,-0.422,-0.484
279,117,House,21715,13,13,NY,100,NaN,NaN,"ESPAILLAT, Adriano J.",E000297,1954,NaN,-0.614,-0.238,-15.18338,0.97922,723.0,7.0,NaN,-0.535,-0.527


In [4]:
members_vv.query("chamber == 'Senate' & born >= 1950 & born <= 1959")

,congress,chamber,icpsr,state_icpsr,district_code,state_abbrev,party_code,occupancy,last_means,bioname,bioguide_id,born,died,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,conditional,nokken_poole_dim1,nokken_poole_dim2
453,117,Senate,42102,41,0,AL,200,NaN,NaN,"TUBERVILLE, Thomas Hawley (Tommy)",T000278,1954,NaN,0.936,-0.352,-75.54402,0.90957,797.0,28.0,NaN,0.874,-0.213
455,117,Senate,40300,81,0,AK,200,NaN,NaN,"MURKOWSKI, Lisa",M001153,1957,NaN,0.210,-0.302,-125.63009,0.84745,759.0,57.0,NaN,0.209,-0.365
459,117,Senate,20101,42,0,AR,200,NaN,NaN,"BOOZMAN, John",B001236,1950,NaN,0.426,0.323,-159.12798,0.82144,809.0,75.0,NaN,0.576,0.303
465,117,Senate,42101,62,0,CO,100,NaN,NaN,"HICKENLOOPER, John Wright",H000273,1952,NaN,-0.238,-0.024,-15.31544,0.98136,814.0,5.0,NaN,-0.238,-0.025
471,117,Senate,41903,43,0,FL,200,NaN,NaN,"SCOTT, Richard Lynn (Rick)",S001217,1952,NaN,0.650,0.183,-90.31931,0.89299,798.0,40.0,NaN,0.719,0.182
477,117,Senate,29345,63,0,ID,200,NaN,NaN,"CRAPO, Michael Dean",C000880,1951,NaN,0.498,0.276,-166.50373,0.81336,806.0,88.0,NaN,0.438,0.398
482,117,Senate,41900,22,0,IN,200,NaN,NaN,"BRAUN, Michael",B001310,1954,NaN,0.800,0.600,-72.45621,0.91268,793.0,31.0,NaN,0.749,0.367
486,117,Senate,29722,32,0,KS,200,NaN,NaN,"MORAN, Jerry",M000934,1954,NaN,0.418,0.229,-131.44570,0.83604,734.0,56.0,NaN,0.413,0.358
489,117,Senate,20919,45,0,LA,200,NaN,NaN,"CASSIDY, Bill",C001075,1957,NaN,0.465,-0.066,-191.67279,0.78481,791.0,84.0,NaN,0.475,-0.030
490,117,Senate,41703,45,0,LA,200,NaN,NaN,"KENNEDY, John Neely",K000393,1951,NaN,0.572,0.012,-183.59519,0.78639,764.0,73.0,NaN,0.583,-0.080


bills_117.groupby('sponsor_name').size().sort_values(ascending=False)

bills_117['house_passage'][0] is not None

bills_117['pass_house'] = [x is not None for x in bills_117['house_passage']]
bills_117['pass_senate'] = [x is not None for x in bills_117['senate_passage']]
bills_117['pass_enact'] = [x is not None for x in bills_117['enacted']]

bills_117.groupby('sponsor_party').agg({'pass_house': 'mean',
                                       'pass_senate':'mean',
                                       'pass_enact':'mean'})

In [5]:
members_vv.head(3)

,congress,chamber,icpsr,state_icpsr,district_code,state_abbrev,party_code,occupancy,last_means,bioname,bioguide_id,born,died,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,conditional,nokken_poole_dim1,nokken_poole_dim2
0,117,President,99912,99,0,USA,200,0.0,0.0,"TRUMP, Donald John",NaN,1946,NaN,0.403,0.162,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,117,President,99913,99,0,USA,100,0.0,0.0,"BIDEN, Joseph Robinette, Jr.",B000444,1942,NaN,-0.320,0.283,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,117,House,20301,41,3,AL,200,NaN,NaN,"ROGERS, Mike Dennis",R000575,1958,NaN,0.360,0.463,-74.23598,0.89924,699.0,24.0,NaN,0.422,0.539


In [6]:
house_pp.head(3)

,id,title,short_title,api_uri,first_name,middle_name,last_name,suffix,date_of_birth,gender,party,leadership_role,twitter_account,facebook_account,youtube_account,govtrack_id,cspan_id,votesmart_id,icpsr_id,crp_id,google_entity_id,fec_candidate_id,url,rss_url,contact_form,in_office,cook_pvi,dw_nominate,ideal_point,seniority,next_election,total_votes,missed_votes,total_present,last_updated,ocd_id,office,phone,fax,state,district,at_large,geoid,missed_votes_pct,votes_with_party_pct,votes_against_party_pct
0,A000370,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Alma,None,Adams,None,1946-05-27,F,D,None,RepAdams,CongresswomanAdams,None,412607,76386,5935,21545,N00035451,/m/02b45d,H4NC12100,https://adams.house.gov,https://adams.house.gov/rss.xml,None,True,None,-0.465,None,10,2022,925.0,3.0,0.0,2022-10-13 09:30:11 -0400,ocd-division/country:us/state:nc/cd:12,2436 Rayburn House Office Building,202-225-1510,None,NC,12,False,3712,0.32,99.02,0.87
1,A000055,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Robert,B.,Aderholt,None,1965-07-22,M,R,None,Robert_Aderholt,RobertAderholt,RobertAderholt,400004,45516,441,29701,N00003028,/m/024p03,H6AL04098,https://aderholt.house.gov,https://aderholt.house.gov/rss.xml,None,True,None,0.380,None,26,2022,925.0,14.0,0.0,2022-10-13 09:30:10 -0400,ocd-division/country:us/state:al/cd:4,266 Cannon House Office Building,202-225-4876,None,AL,4,False,0104,1.51,96.24,3.65
2,A000371,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Pete,None,Aguilar,None,1979-06-19,M,D,House Democratic Caucus Vice Chair,reppeteaguilar,reppeteaguilar,None,412615,79994,70114,21506,N00033997,/m/0jwv0xf,H2CA31125,https://aguilar.house.gov,None,None,True,None,-0.296,None,8,2022,925.0,3.0,0.0,2022-10-13 09:30:11 -0400,ocd-division/country:us/state:ca/cd:31,109 Cannon House Office Building,202-225-3201,None,CA,31,False,0631,0.32,98.80,1.09


In [7]:
senate_pp.head(3)

,id,title,short_title,api_uri,first_name,middle_name,last_name,suffix,date_of_birth,gender,party,leadership_role,twitter_account,facebook_account,youtube_account,govtrack_id,cspan_id,votesmart_id,icpsr_id,crp_id,google_entity_id,fec_candidate_id,url,rss_url,contact_form,in_office,cook_pvi,dw_nominate,ideal_point,seniority,next_election,total_votes,missed_votes,total_present,last_updated,ocd_id,office,phone,fax,state,senate_class,state_rank,lis_id,missed_votes_pct,votes_with_party_pct,votes_against_party_pct
0,B001230,"Senator, 1st Class",Sen.,https://api.propublica.org/congress/v1/members...,Tammy,None,Baldwin,None,1962-02-11,F,D,Senate Democratic Caucus Secretary,SenatorBaldwin,senatortammybaldwin,witammybaldwin,400013,57884,3470,29940,N00004367,/m/024v02,H8WI00018,https://www.baldwin.senate.gov,None,https://www.baldwin.senate.gov/feedback,True,None,-0.493,None,9,2024,866,1,0,2022-10-01 01:45:44 -0400,ocd-division/country:us/state:wi,709 Hart Senate Office Building,202-224-5653,None,WI,1,junior,S354,0.12,98.96,1.04
1,B001261,"Senator, 1st Class",Sen.,https://api.propublica.org/congress/v1/members...,John,None,Barrasso,None,1952-07-21,M,R,Senate Republican Conference Chair,SenJohnBarrasso,johnbarrasso,barrassowyo,412251,1024777,52662,40707,N00006236,/m/02rsm32,S6WY00068,https://www.barrasso.senate.gov,https://www.barrasso.senate.gov/public/?a=rss....,https://www.barrasso.senate.gov/public/index.c...,True,None,0.537,None,15,2024,866,58,0,2022-10-01 01:45:43 -0400,ocd-division/country:us/state:wy,307 Dirksen Senate Office Building,202-224-6441,None,WY,1,senior,S317,6.70,95.33,4.67
2,B001267,"Senator, 3rd Class",Sen.,https://api.propublica.org/congress/v1/members...,Michael,None,Bennet,None,1964-11-28,M,D,None,SenBennetCo,senbennetco,SenatorBennet,412330,1031622,110942,40910,N00030608,/m/05b60qf,S0CO00211,https://www.bennet.senate.gov/public,None,https://www.bennet.senate.gov/public/index.cfm...,True,None,-0.229,None,13,2022,866,19,0,2022-10-12 09:45:23 -0400,ocd-division/country:us/state:co,261 Russell Senate Office Building,202-224-5852,202-228-5097,CO,3,senior,S330,2.19,99.06,0.94


In [8]:
members_pp = pd.concat([house_pp, senate_pp], ignore_index=True)
members_pp.head(3)

,id,title,short_title,api_uri,first_name,middle_name,last_name,suffix,date_of_birth,gender,party,leadership_role,twitter_account,facebook_account,youtube_account,govtrack_id,cspan_id,votesmart_id,icpsr_id,crp_id,google_entity_id,fec_candidate_id,url,rss_url,contact_form,in_office,cook_pvi,dw_nominate,ideal_point,seniority,next_election,total_votes,missed_votes,total_present,last_updated,ocd_id,office,phone,fax,state,district,at_large,geoid,missed_votes_pct,votes_with_party_pct,votes_against_party_pct,senate_class,state_rank,lis_id
0,A000370,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Alma,None,Adams,None,1946-05-27,F,D,None,RepAdams,CongresswomanAdams,None,412607,76386,5935,21545,N00035451,/m/02b45d,H4NC12100,https://adams.house.gov,https://adams.house.gov/rss.xml,None,True,None,-0.465,None,10,2022,925.0,3.0,0.0,2022-10-13 09:30:11 -0400,ocd-division/country:us/state:nc/cd:12,2436 Rayburn House Office Building,202-225-1510,None,NC,12,False,3712,0.32,99.02,0.87,NaN,NaN,NaN
1,A000055,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Robert,B.,Aderholt,None,1965-07-22,M,R,None,Robert_Aderholt,RobertAderholt,RobertAderholt,400004,45516,441,29701,N00003028,/m/024p03,H6AL04098,https://aderholt.house.gov,https://aderholt.house.gov/rss.xml,None,True,None,0.380,None,26,2022,925.0,14.0,0.0,2022-10-13 09:30:10 -0400,ocd-division/country:us/state:al/cd:4,266 Cannon House Office Building,202-225-4876,None,AL,4,False,0104,1.51,96.24,3.65,NaN,NaN,NaN
2,A000371,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Pete,None,Aguilar,None,1979-06-19,M,D,House Democratic Caucus Vice Chair,reppeteaguilar,reppeteaguilar,None,412615,79994,70114,21506,N00033997,/m/0jwv0xf,H2CA31125,https://aguilar.house.gov,None,None,True,None,-0.296,None,8,2022,925.0,3.0,0.0,2022-10-13 09:30:11 -0400,ocd-division/country:us/state:ca/cd:31,109 Cannon House Office Building,202-225-3201,None,CA,31,False,0631,0.32,98.80,1.09,NaN,NaN,NaN


In [9]:
members_pp.query("icpsr_id == '20301'")

,id,title,short_title,api_uri,first_name,middle_name,last_name,suffix,date_of_birth,gender,party,leadership_role,twitter_account,facebook_account,youtube_account,govtrack_id,cspan_id,votesmart_id,icpsr_id,crp_id,google_entity_id,fec_candidate_id,url,rss_url,contact_form,in_office,cook_pvi,dw_nominate,ideal_point,seniority,next_election,total_votes,missed_votes,total_present,last_updated,ocd_id,office,phone,fax,state,district,at_large,geoid,missed_votes_pct,votes_with_party_pct,votes_against_party_pct,senate_class,state_rank,lis_id
345,R000575,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Mike,D.,Rogers,None,1958-07-16,M,R,None,RepMikeRogersAL,CongressmanMikeDRogers,MikeRogersAL03,400341,1014740,5705,20301,N00024759,/m/024n_s,H2AL03032,https://mikerogers.house.gov,None,None,True,None,0.36,None,22,2022,925.0,37.0,0.0,2022-10-13 09:30:10 -0400,ocd-division/country:us/state:al/cd:3,2469 Rayburn House Office Building,202-225-3261,None,AL,3,False,0103,4.0,96.6,3.28,NaN,NaN,NaN


In [16]:
members_pp['icpsr'] = members_pp['icpsr_id'].astype('float')

In [18]:
members_total = pd.merge(members_vv, members_pp,
                        on = 'icpsr',
                        how = 'outer',
                        #validate = 'one_to_one',
                        indicator = 'matched')

In [19]:
members_total['matched'].value_counts()

both          385
right_only    171
left_only     170
Name: matched, dtype: int64

In [22]:
pd.set_option('display.max_columns', 80)
members_total.query("matched == 'right_only'")[members_pp.columns]

,id,title,short_title,api_uri,first_name,middle_name,last_name,suffix,date_of_birth,gender,party,leadership_role,twitter_account,facebook_account,youtube_account,govtrack_id,cspan_id,votesmart_id,icpsr_id,crp_id,google_entity_id,fec_candidate_id,url,rss_url,contact_form,in_office,cook_pvi,dw_nominate,ideal_point,seniority,next_election,total_votes,missed_votes,total_present,last_updated,ocd_id,office,phone,fax,state,district,at_large,geoid,missed_votes_pct,votes_with_party_pct,votes_against_party_pct,senate_class,state_rank,lis_id,icpsr
555,A000376,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Colin,None,Allred,None,1983-04-15,M,D,None,RepColinAllred,None,None,412828,None,177357,None,N00040989,/m/03d066b,H8TX32098,https://allred.house.gov,None,None,True,None,NaN,None,4,2022,925.0,10.0,0.0,2022-10-13 09:30:10 -0400,ocd-division/country:us/state:tx/cd:32,114 Cannon House Office Building,202-225-2231,None,TX,32,False,4832,1.08,98.14,1.75,NaN,NaN,NaN,NaN
556,A000377,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Kelly,None,Armstrong,None,1976-10-08,M,R,None,RepArmstrongND,None,None,412794,None,139338,None,N00042868,/g/11hcszksh3,H8ND00096,https://armstrong.house.gov,None,None,True,None,NaN,None,4,2022,925.0,47.0,1.0,2022-10-13 09:30:10 -0400,ocd-division/country:us/state:nd/cd:1,1740 Longworth House Office Building,202-225-2611,None,ND,At-Large,True,3800,5.08,95.64,4.24,NaN,NaN,NaN,NaN
557,A000148,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Jake,None,Auchincloss,None,1988-01-29,M,D,None,RepAuchincloss,None,None,456825,None,None,None,N00045506,/g/11f0xw2_p4,H0MA04192,https://auchincloss.house.gov,None,None,True,None,NaN,None,2,2022,925.0,3.0,0.0,2022-10-13 09:30:11 -0400,ocd-division/country:us/state:ma/cd:4,1524 Longworth House Office Building,202-225-5931,None,MA,4,False,2504,0.32,99.13,0.76,NaN,NaN,NaN,NaN
558,A000378,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Cynthia,None,Axne,None,1965-04-20,F,D,None,RepCindyAxne,None,None,412772,None,179226,None,N00041104,/g/11fgktcgcb,H8IA03124,https://axne.house.gov,None,None,True,None,NaN,None,4,2022,925.0,1.0,0.0,2022-10-13 09:30:10 -0400,ocd-division/country:us/state:ia/cd:3,1034 Longworth House Office Building,202-225-5476,None,IA,3,False,1903,0.11,97.51,2.39,NaN,NaN,NaN,NaN
559,B001307,Representative,Rep.,https://api.propublica.org/congress/v1/members...,James,None,Baird,None,1945-06-04,M,R,None,RepJimBaird,None,None,412777,None,86013,None,N00041954,/g/11gfd5wndr,H8IN04199,https://baird.house.gov,None,None,True,None,NaN,None,4,2022,925.0,3.0,0.0,2022-10-13 09:30:10 -0400,ocd-division/country:us/state:in/cd:4,1314 Longworth House Office Building,202-225-5037,None,IN,4,False,1804,0.32,96.62,3.28,NaN,NaN,NaN,NaN
560,B001306,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Troy,None,Balderson,None,1962-01-16,M,R,None,RepBalderson,None,None,412747,None,102781,None,N00042194,/m/09ry4tm,H8OH12180,https://balderson.house.gov,None,None,True,None,NaN,None,6,2022,925.0,10.0,0.0,2022-10-13 09:30:10 -0400,ocd-division/country:us/state:oh/cd:12,2429 Rayburn House Office Building,202-225-5355,None,OH,12,False,3912,1.08,97.36,2.53,NaN,NaN,NaN,NaN
561,B000668,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Cliff,None,Bentz,None,1952-01-12,M,R,None,RepBentz,None,None,456842,None,None,None,N00045773,/m/0gfdsjb,H0OR02127,https://bentz.house.gov,None,None,True,None,NaN,None,2,2022,925.0,4.0,0.0,2022-10-13 09:30:10 -0400,ocd-division/country:us/state:or/cd:2,1239 Longworth House Office Building,202-225-6730,None,OR,2,False,4102,0.43,97.05,2.84,NaN,NaN,NaN,NaN
562,B000740,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Stephanie,None,Bice,None,1973-11-11,F,R,None,RepBice,None,None,456841,None,None,None,N00044579,/g/11h2hmkrv8,H0OK05205,https://bice.house.gov,None,None,True,None,NaN,None,2,2022,925.0,3.0,0.0,2022-10-13 09:30:10 -0400,ocd-division/

In [24]:
members_pp.columns

Index(['id', 'title', 'short_title', 'api_uri', 'first_name', 'middle_name',
       'last_name', 'suffix', 'date_of_birth', 'gender', 'party',
       'leadership_role', 'twitter_account', 'facebook_account',
       'youtube_account', 'govtrack_id', 'cspan_id', 'votesmart_id',
       'icpsr_id', 'crp_id', 'google_entity_id', 'fec_candidate_id', 'url',
       'rss_url', 'contact_form', 'in_office', 'cook_pvi', 'dw_nominate',
       'ideal_point', 'seniority', 'next_election', 'total_votes',
       'missed_votes', 'total_present', 'last_updated', 'ocd_id', 'office',
       'phone', 'fax', 'state', 'district', 'at_large', 'geoid',
       'missed_votes_pct', 'votes_with_party_pct', 'votes_against_party_pct',
       'senate_class', 'state_rank', 'lis_id', 'icpsr'],
      dtype='object')

In [25]:
members_vv.columns

Index(['congress', 'chamber', 'icpsr', 'state_icpsr', 'district_code',
       'state_abbrev', 'party_code', 'occupancy', 'last_means', 'bioname',
       'bioguide_id', 'born', 'died', 'nominate_dim1', 'nominate_dim2',
       'nominate_log_likelihood', 'nominate_geo_mean_probability',
       'nominate_number_of_votes', 'nominate_number_of_errors', 'conditional',
       'nokken_poole_dim1', 'nokken_poole_dim2'],
      dtype='object')

In [33]:
members_pp.groupby(['last_name', 'date_of_birth']).sort_values(ascending=False)

AttributeError: 'DataFrameGroupBy' object has no attribute 'sort_values'

In [27]:
members_vv[['bioname', 'born']].head(5)

,bioname,born
0,"TRUMP, Donald John",1946
1,"BIDEN, Joseph Robinette, Jr.",1942
2,"ROGERS, Mike Dennis",1958
3,"SEWELL, Terri",1965
4,"BROOKS, Mo",1954


In [29]:
members_pp[['last_name', 'date_of_birth']].head(5)

,last_name,date_of_birth
0,Adams,1946-05-27
1,Aderholt,1965-07-22
2,Aguilar,1979-06-19
3,Allen,1951-11-07
4,Allred,1983-04-15


1. Extract last name from bioname in members_vv
2. Convert last_name in members_pp to all caps
3. Extract the year from date_of_birth in members_pp
4. Check whether (last name, year) is unique in both members_vv and members_pp
5. If so, merge on (last name, year); If not, try with (last name, year, state)

In [40]:
members_vv['last_name'] = members_vv['bioname'].str.split(pat=',', expand=True)[0]

In [67]:
members_pp['last_name'] = members_pp['last_name'].str.upper()
members_vv['last_name'] = members_vv['last_name'].str.upper()

In [49]:
members_pp['born'] = members_pp['date_of_birth'].str.split(pat='-', expand=True)[0]

In [53]:
members_vv[['last_name', 'born', 'state_abbrev']].duplicated().value_counts()

False    555
dtype: int64

In [54]:
members_pp[['last_name', 'born', 'state']].duplicated().value_counts()

False    556
dtype: int64

In [56]:
members_vv['born'] = members_vv['born'].astype('str')
members_pp['born'] = members_pp['born'].astype('str')

In [68]:
members_total = pd.merge(members_vv, members_pp,
                        left_on = ['last_name', 'born', 'state_abbrev'],
                        right_on = ['last_name', 'born', 'state'],
                        how = 'outer',
                        validate = 'one_to_one',
                        indicator = 'matched')

In [69]:
members_total['matched'].value_counts()

both          547
right_only      9
left_only       8
Name: matched, dtype: int64

In [70]:
members_total.query("matched == 'right_only'")[['last_name', 'born', 'state']]

,last_name,born,state
555,FERNANDEZ,1959,NM
556,FULCHER,1973,ID
557,GIMÉNEZ,1954,FL
558,GONZÁLEZ-COLÓN,1976,PR
559,OMAR,1981,MN
560,PELTOLA,1973,AK
561,PFLUGER,1978,TX
562,RYAN,1982,NY
563,SEMPOLINSKI,1983,NY


In [71]:
members_total.query("matched == 'left_only'")[['last_name', 'born', 'state_abbrev']]

,last_name,born,state_abbrev
0,TRUMP,1946,USA
1,BIDEN,1942,USA
116,GIMENEZ,1954,FL
137,FULCHER,1962,ID
224,OMAR,1982,MN
266,LEGER FERNANDEZ,1959,NM
356,GONZÁLEZ-CÓLON,1976,PR
404,PFLUGER,1977,TX
